In [ ]:
import random
random.seed(123)

## IC Performance

In [3]:
import numpy as np
import pandas
from tqdm import tqdm

# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.05
Le = 2.23
L_SSGR =  3
# df = 5
df = 20
# shape_param = 1  # Shape parameter for the gamma distribution
shape_param = 6.058  # Shape parameter for the gamma distribution
scale_param = 0.33  # Scale parameter for the gamma distribution

# Control Limits
LCL = exp_w - Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))
UCL = exp_w + Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))

# Simulating Phase I and Phase II
run_lengths = []
previous_run_length = None  # Store the previous nonconforming run length
total_runs = []
for i in tqdm(range(sim)):
    zi_1 = exp_w  # Phase I mean as starting EWMA value
    # xi = np.random.normal(0, 1, m)  # Phase I data Normal
    # xi = np.random.standard_t(df, m)  # Phase I data t
    xi = np.random.gamma(shape_param, scale_param, m) + 5.18 # Phase I data Gamma
    # xi=(xi-shape_param)/np.sqrt(shape_param)
    count = 0
    signaled = False
    total_run = 0
    while not signaled:
        count += 1
        total_run += 1
        # yi = np.random.normal(0, 1, n)  # Phase II data
        # yi = np.random.standard_t(df, n)  # Phase II data t
        yi = np.random.gamma(shape_param, scale_param, n) + 5.18  # Phase II data Gamma
        # yi=(yi-shape_param)/np.sqrt(shape_param)
        comb = np.concatenate((xi, yi))
        ranks = np.argsort(np.argsort(comb)) + 1  # Rank calculation
        W = ranks[-n:].sum()  # Sum of ranks for new sample

        # Calculate W-EWMA
        zi = lambda_ * W + (1 - lambda_) * zi_1
        zi_1 = zi
        # Check control limits and evaluate run length
        if zi >= UCL or zi <= LCL:
            if previous_run_length is None:
                # Check if the first nonconforming group is below the threshold
                if count <= L_SSGR:
                    run_lengths.append(count)
                    total_runs.append(total_run)
                    signaled = True
            elif previous_run_length <= L_SSGR and count <= L_SSGR:
                run_lengths.append(count)
                total_runs.append(total_run)
                signaled = True

            previous_run_length = count
            count = 0  # Reset count after nonconforming group and no signal

run_lengths = np.array(total_runs)
print(f"Mean Run Length: {run_lengths.mean()}")
print(f"Standard Deviation: {run_lengths.std()}")
print(f"5th Percentile: {np.percentile(run_lengths, 5)}")
print(f"95th Percentile: {np.percentile(run_lengths, 95)}")
print(f"Median: {np.percentile(run_lengths, 50)}")


100%|██████████| 10000/10000 [01:01<00:00, 162.55it/s]

Mean Run Length: 193.4022
Standard Deviation: 264.9671780337331
5th Percentile: 17.0
95th Percentile: 700.0
Median: 97.0


## With shifts

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
# np.random.seed(123)
# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = np.sqrt(n * m * (n + m + 1) / 12)
lambda_ = 0.5
Le = 1.863       # For lamba=0.5
# Le = 2.234         # For lambda=0.05
L_SSGR = 3
df = 5
# df = 20
# shape_param = 1  # Shape parameter for the gamma distribution
shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution

# Control Limits
LCL = exp_w - Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))
UCL = exp_w + Le * stdev_w * np.sqrt(lambda_ / (2 - lambda_))

# Delta values
deltas = np.arange(0, 3.25, 0.25)

# DataFrame to store results
results = pd.DataFrame(columns=['Delta', 'ARL', 'SDRL', 'MDRL'])
previous_run_length = None

for delta in deltas:
    print(delta)
    total_runs = []
    run_lengths = []
    previous_run_length = None
    for i in tqdm(range(sim)):
        zi_1 = exp_w  # Phase I mean as starting EWMA value
        # xi = np.random.normal(0, 1, m)  # Phase I data Normal
        xi = np.random.standard_t(df, m)  # Phase I data t
        # xi = np.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
        # xi=(xi-shape_param)/np.sqrt(shape_param)
        count = 0
        signaled = False
        total_run = 0
        while not signaled:
            count += 1
            total_run += 1
            # yi = np.random.normal(delta, 1, n)  # Phase II data
            yi = np.random.standard_t(df, n) + delta  # Phase II data t
            # y1 = np.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
            # yi=(y1-shape_param)/np.sqrt(shape_param) + delta
            comb = np.concatenate((xi, yi))
            ranks = np.argsort(np.argsort(comb)) + 1
            W = ranks[-n:].sum()

            zi = lambda_ * W + (1 - lambda_) * zi_1  # Calculate W-EWMA
            zi_1 = zi

            if zi >= UCL or zi <= LCL:
                if previous_run_length is None:
                # Check if the first nonconforming group is below the threshold
                    if count <= L_SSGR:
                        run_lengths.append(count)
                        total_runs.append(total_run)
                        signaled = True
                elif previous_run_length <= L_SSGR and count <= L_SSGR:
                    run_lengths.append(count)
                    total_runs.append(total_run)
                    signaled = True
                    
                previous_run_length = count
                count = 0  # Reset count after nonconforming group

    # Calculate metrics
    run_lengths = np.array(total_runs)
    ARL = run_lengths.mean()
    SDRL = run_lengths.std()
    MDRL = np.percentile(run_lengths, 50)

    # Append to results
    results_row = pd.DataFrame({'Delta': [delta], 'ARL': [ARL], 'SDRL': [SDRL], 'MDRL': [MDRL]})
    results = pd.concat([results, results_row], ignore_index=True)

results


# Multiple Execution

In [ ]:
import numpy as numpy
import pandas as pandas
from tqdm import tqdm

# Parameters
m = 100
sim = 10000
n = 5
exp_w = (n * (n + m + 1)) / 2
stdev_w = numpy.sqrt(n * m * (n + m + 1) / 12)
L_SSGR = 3
# df = 5
df = 20
# shape_param = 1  # Shape parameter for the gamma distribution
shape_param = 15  # Shape parameter for the gamma distribution
scale_param = 1  # Scale parameter for the gamma distribution

# Delta values
deltas = numpy.arange(0, 3.25, 0.25)

# Parameters for lambda and Le
lambda_values = [0.05, 0.5]
Le_values = [2.234, 1.863]

# DataFrame to store results
results = pandas.DataFrame(columns=['Delta', 'ARL (0.5)', 'ARL (0.05)'])

for lambda_, Le in zip(lambda_values, Le_values):
    # Control Limits
    LCL = exp_w - Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    UCL = exp_w + Le * stdev_w * numpy.sqrt(lambda_ / (2 - lambda_))
    
    column_name = f'ARL ({lambda_})'
    
    for delta in deltas:
        print(f"Delta: {delta}, Lambda: {lambda_}")
        total_runs = []
        run_lengths = []
        previous_run_length = None
        
        for i in tqdm(range(sim)):
            zi_1 = exp_w  # Phase I mean as starting EWMA value
            
            # Phase I data (t-distribution standardized to zero mean and unit variance)
            # xi = numpy.random.normal(0, 1, m)  # Phase I data Normal
            # xi = numpy.random.standard_t(df, m)  # Phase I data t
            xi = numpy.random.gamma(shape_param, scale_param, m)  # Phase I data Gamma
            xi=(xi-shape_param)/numpy.sqrt(shape_param)
            
            count = 0
            signaled = False
            total_run = 0
            
            while not signaled:
                count += 1
                total_run += 1
                
                # Phase II data (t-distribution standardized and shifted by delta)
                # yi = numpy.random.normal(delta, 1, n)  # Phase II data
                # yi = numpy.random.standard_t(df, n) + delta  # Phase II data t
                y1 = numpy.random.gamma(shape_param, scale_param, n)  # Phase II data Gamma
                yi=(y1-shape_param)/numpy.sqrt(shape_param) + delta
                
                comb = numpy.concatenate((xi, yi))
                ranks = numpy.argsort(numpy.argsort(comb)) + 1
                W = ranks[-n:].sum()
    
                zi = lambda_ * W + (1 - lambda_) * zi_1  # Calculate W-EWMA
                zi_1 = zi
    
                if zi >= UCL or zi <= LCL:
                    if previous_run_length is None:
                        # Check if the first nonconforming group is below the threshold
                        if count <= L_SSGR:
                            run_lengths.append(count)
                            total_runs.append(total_run)
                            signaled = True
                    elif previous_run_length <= L_SSGR and count <= L_SSGR:
                        run_lengths.append(count)
                        total_runs.append(total_run)
                        signaled = True
    
                    previous_run_length = count
                    count = 0  # Reset count after nonconforming group
    
        # Calculate metrics
        run_lengths = numpy.array(total_runs)
        ARL = run_lengths.mean()
        
        # Store results
        if delta in results['Delta'].values:
            results.loc[results['Delta'] == delta, column_name] = ARL
        else:
            new_row = pandas.DataFrame({'Delta': [delta], column_name: [ARL]})
            results = pandas.concat([results, new_row], ignore_index=True)

# Format the columns as (ARL_0.5, ARL_0.05)
results['ARL'] = results.apply(lambda row: f"({row['ARL (0.05)']:.2f}, {row['ARL (0.5)']:.2f})", axis=1)
results = results[['Delta', 'ARL']]

results
